# Basic tutorial: Benchmarking
#### Author: Abdul Jabbar

This short tutorial provides you with the basic functioning of benchmarking.

The main steps of the tutorial are the following:
 1. creation of a datasets
 2. creation of models
 3. define metrics and losses
 4. run benchmark

In [11]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

from gdeep.models import FFNet
from gdeep.data import TorchDataLoader

import optuna

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [12]:
writer = SummaryWriter()

# Create your datasets dictionaries

In [13]:
from torch.utils.data.sampler import SubsetRandomSampler

dataloaders_dicts = []
dl = TorchDataLoader(name="CIFAR10")
train_indices = []
for i in range(5120):
    train_indices.append(i)

dl_tr, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(1024):
    test_indices.append(i)

dl_ts, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_5000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)
print(temp_dict["name"], "created")

train_indices = []
for i in range(10240):
    train_indices.append(i)

dl_tr, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(2048):
    test_indices.append(i)

dl_ts, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_10000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)
print(temp_dict["name"], "created")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
CIFAR10_5000 created
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
CIFAR10_10000 created


## Define models dictionaries

In [14]:
import torchvision.models as models

models_dicts = []

model = nn.Sequential(models.resnet18(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "resnet18"
temp_dict["model"] = model

models_dicts.append(temp_dict)
print(temp_dict["name"], "created")

model = nn.Sequential(models.vgg16(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "vgg16"
temp_dict["model"] = model

models_dicts.append(temp_dict)
print(temp_dict["name"], "created")

resnet18 created
vgg16 created


## Define hyperparamters

In [15]:
from torch.optim import SGD

loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = SGD

## Benchmarking

In [16]:
from gdeep.search.benchmark import Benchmark

bench = Benchmark(models_dicts, dataloaders_dicts, loss_fn, writer)

bench.start(optimizer, epochs = 1, batch_size = 1024, lr = learning_rate)

Benchmarking Started
******************************
Training on Dataset: CIFAR10_5000, Model: resnet18
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
TOTAL EPOCHS  1
Epoch 1
-------------------------------


d:\anaconda3\envs\giotto-deep2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training loss: 1.454203  [40/40]
Time taken for this epoch: 25s
Validation results: 
 Accuracy: 9.0%,                 Avg loss: 0.000328 

Test results: 
 Accuracy: 0.9%,                 Avg loss: 0.000034 

Done!
******************************
Training on Dataset: CIFAR10_5000, Model: vgg16
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
TOTAL EPOCHS  1
Epoch 1
-------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 6.00 GiB total capacity; 1.91 GiB already allocated; 2.15 GiB free; 1.99 GiB reserved in total by PyTorch)